In [1]:
import numpy as np  
import matplotlib.pyplot as plt  
import astropy.constants as c  
import astropy.units as u  

year  = (1*u.year).cgs.value
au    = c.au.cgs.value
M_jup = c.M_jup.cgs.value
M_sun = c.M_sun.cgs.value

from simple_slider import Widget
from simple_slider import kanagawa_profile
from simple_slider import get_surface_density
from simple_slider import get_disk_height

%matplotlib notebook

In [2]:
import simple_slider
import os

In [3]:
data_dir = simple_slider.pkg_resources.resource_filename(simple_slider.__name__, 'data_1_planet')
r = (np.loadtxt(os.path.join(data_dir, 'radius.dat')))
sigma = np.loadtxt(os.path.join(data_dir, 'sigma_averaged.dat'), unpack=1)
time = np.loadtxt(os.path.join(data_dir, 'time.dat'))

Get the data

In [4]:
t   = time.searchsorted(5 * 1e5 * year)
#t  = 0
sig = sigma[:, t]
print(len(r))

1024


Define the logp function: create a model based on the parameters and compare it to the data

In [5]:
from log_prob import logp, log_prior, log_prob

In [ ]:
def conv_params_model(params, x_data):
    
    n_planets = int((params.shape[0]-3)/2)
    
    x_min = np.log10(x_data[0]/ au) 
    x_max = np.log10(x_data[-1]/ au) 
    
    # convert walkers from {0,1} to physical values for now    
    params[0] = 10**(params[0]*3 -4)
    params[1] = 10**(4 * (params[1]-0.5))
    params[2] = (2.0 * params[2]) - 2
    for n in range(n_planets):
        params[3+2*n] = 10**(params[3 + 2 * n] * (x_max - x_min) + x_min) * au
        params[4+2*n] = 10**(params[4 + 2 * n] * 2.0 - 4.0)
        
    #h_p = np.interp(R_p, x_data, get_disk_height(x_data))
    
    return params

In [6]:
import emcee
from multiprocessing import Pool

nwalkers = 100
# need to input ndim as 3 + (2 * n_plantes) somewhere else
ndim = 5
n_burnin = 250
n_steps = 2000

pos = np.random.rand(nwalkers, ndim)
nwalkers, ndim = pos.shape

with Pool() as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob,args=(r, sig, 1), pool=pool)
    sampler.run_mcmc(pos, n_burnin, progress=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:01<00:00, 157.75it/s]


In [ ]:
sampler.reset()

sampler.run_mcmc(pos, n_steps, progress=True)

In [ ]:
print("Mean acceptance fraction: {0:.3f}".format(np.mean(sampler.acceptance_fraction)))
#print(sampler.get_autocorr_time())

In [ ]:
sampler.lnprobability.max()

In [ ]:
sampler.lnprobability.shape

In [ ]:
f, ax = plt.subplots()
ax.hist(np.log10(-sampler.lnprobability[:, -1]))

In [ ]:
f, ax = plt.subplots(figsize=(10, 5))
ax.semilogy(np.arange(sampler.lnprobability.shape[-1]), -sampler.lnprobability.T)

In [ ]:
fig, axes = plt.subplots(ndim, figsize=(12, 8), sharex=True)
samples = sampler.get_chain()
labels = ["alpha", "sig0", "p", "R_p","mass_ratios", "R_p","mass_ratios", "R_p","mass_ratios"]
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
flat_samples = sampler.get_chain(discard=200, flat=True)
means = []
mcmc = []
for i in range(ndim):
    means += [np.mean(flat_samples[:,i])]
    mcmc += [np.percentile(flat_samples[:, i], 50)]
    
print("mean values: ", means)
print("50th percentiles: ", mcmc)

In [ ]:
#labels = ["alpha", "sig0", "p", "R_p", "mass_ratios"]
print(flat_samples.shape)

import corner
fig = corner.corner(flat_samples, labels=labels, quantiles=[0.16, 0.5, 0.84]);

def conv_params_model(params, x_data, n_planets):
    
    x_min = np.log10(x_data[0]/ au) 
    x_max = np.log10(x_data[-1]/ au) 
    
    # convert walkers from {0,1} to physical values for now    
    alpha = 10**(params[0]*3 -4)
    sig0  = 10**(4 * (params[1]-0.5))
    p     = (2.0 * params[2]) - 2
    R_p   = []
    mass_ratios = []
    for n in range(n_planets):
        R_p         += [10**(params[3 + 3 * n] * (x_max - x_min) + x_min) * au]
        mass_ratios += [10**(params[4 + 3 * n] * 2.0 - 4.0)]
        
    h_p = np.interp(R_p, x_data, get_disk_height(x_data))
    
    return alpha, sig0, p, R_p, h_p, mass_ratios

In [ ]:
inds = np.random.randint(len(flat_samples), size=100)
f, ax2 = plt.subplots(figsize=(8, 8))
for ind in inds:
    sample = flat_samples[ind]
    a, s, p, rp, hp, m = conv_params_model(sample, r, 1)
    ax2.loglog(r, get_surface_density(r, a, s, p, rp, hp, m), "C1", alpha=0.1)
    
a, s, p, rp, hp, m = conv_params_model(mcmc, r, 1)
ax2.loglog(r, get_surface_density(r, a, s, p, rp, hp, m), "b", alpha=0.6)
ax2.loglog(r[0:-1], sig[0:-1], "k")

#print("50th percentiles: alpha: {:.3f}, Sigma: {:.2f}, p: {:.2f}, R_p: {:.3e}, h_p: {:.3e}, mass ratio: {:.2e}".format(a, s, p, rp, hp, m))
print("50th percentile, parameters in cgs units:")
for i in range(ndim):
    print(labels[i], " :", conv_params_model(mcmc, r, 1)[i])
    